# Week 2 - Challenge
## LangChain Practice

### Advanced Customer Service Agent with LangChain

> Deliverables

>> 1. Implementation using LangChain Expression Language (LCEL) to build the agent.
>> 2. Pydantic models for QueryAnalysis and ConversationSummary fully integrated into the chain.
>> 3. Usage examples for each suggested test query, demonstrating the end-to-end functionality.
>> 4. Results analysis. Include a screenshot or public link to a LangSmith trace for one of the complex queries to demonstrate successful tracing.

#### Setup & API Configuration

In [ ]:
# Install packages

!pip install openai
!pip install python-dotenv
!pip install -qU \
  langchain-core \
  langchain-openai \
  langchain-community

In [76]:
# Import libraries

import os
import openai
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate,ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from pydantic import BaseModel, Field
from typing import Literal, Optional, List
import json, datetime

# Load environment variables from .env file

load_dotenv()
openai.api_key  = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

# Define the OpenAI model to use

openai_model = "gpt-4o-mini"

# Initialize the OpenAI chat model

llm = ChatOpenAI(model=openai_model,temperature=0)

In [46]:
test_queries = [
    "Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago"
    ,"Urgent-Negative: This is an emergency! My order #TEC-2024-001 never arrived and I need that laptop for work tomorrow!"
    ,"Satisfied-Positive: Thank you so much for the excellent service with my previous purchase. I want to buy gaming headphones"
    ,"Frustrated-Technical: I can't configure the router I bought last week, I've tried everything and it doesn't work"
    ,"Formal-Billing: Good morning, I need the receipt for my purchase from December 15th, order #TEC-2023-089"
    ,"Warranty-Query: I bought a tablet 8 months ago and now it won't turn on, how do I use the warranty?"
]

#### Pydantic Objects

In [ ]:
# Define the data model for the extracted information

class ExtractedEntities(BaseModel):
    product_name: Optional[str] = Field(None, description="The specific product mentioned by the user")
    order_number: Optional[str] = Field(None, description="The order number mentioned by the user")
    date_info: Optional[str] = Field(None, description="The date mentioned by the user (only if applicable)")

In [ ]:
# Define the query analysis model

class QueryAnalysis(BaseModel):
    """Analyzes and classifies a customer query."""
    query_category: Literal["technical_support","billing","returns","product_inquiry","general_information"]
    urgency_level: Literal["low","medium","high"]
    customer_sentiment: Literal["positive","neutral","negative"]
    entities: ExtractedEntities  

In [ ]:
# Define the prompt template to analyze the user message and extract relevant information

class ConversationSummary(BaseModel):
    """A structured summary of the customer service interaction."""
    timestamp: str
    customer_id: str = "auto_generated"
    conversation_summary: str = Field(description="A concise, one-sentence summary of the interaction.")
    query_category: str
    customer_sentiment: str
    urgency_level: str
    mentioned_products: List[str]
    extracted_information: dict
    resolution_status: Literal["resolved", "pending", "escalated"]
    actions_taken: List[str] = Field(description="A list of actions the agent took or suggested.")
    follow_up_required: bool

#### 1° Component - Query Analysis & Classification

Analyze the initial user query and extract key information into a structured format.

In [71]:
# Model selected

print(llm.model_name)

gpt-4o-mini


In [72]:
# Sample of the first query

user_message = test_queries[0]
user_message

"Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago"

In [ ]:
# Create a specific prompt for query analysis and classification

system_prompt = (
"""
You are an intelligent assistant that analyzes customer service messages and classifies them with the following criteria:

    1. Analyze the customer message and extract relevant information based on the pydantic model "QueryAnalysis".

    2. Be accurate in your classification.

    3. Respond ONLY with valid JSON structure
""")

user_prompt = (
"""
Analyze the customer message: {user_message}
"""
)

analysis_classification = ChatPromptTemplate.from_messages([
system_prompt,
user_prompt
])

# Create the chain

analysis_chain = analysis_classification | llm.with_structured_output(QueryAnalysis)

# Test the chain

test_chain = analysis_chain.invoke({"user_message": user_message})

print(test_chain.model_dump_json(indent=2))

{
  "query_category": "product_inquiry",
  "urgency_level": "medium",
  "customer_sentiment": "neutral",
  "entities": {
    "product_name": "iPhone 15",
    "order_number": null,
    "date_info": null
  }
}


#### 2° Component - Dynamic Response Generation

Generate a context-aware, personalized response.

In [45]:
query_category_prompts = [
    "technical_support: should be empathetic, patient, and ask for relevant troubleshooting steps",
    "billing: should be clear and polite, asking for details like invoice number or payment method",
    "returns: should be understanding and explain the return process, asking for order number if needed",
    "product_inquiry: should be helpful and informative, providing accurate product details",
    "general_information: should be courteous and provide concise, relevant information or guidance"
]

#### 3° Component - Conversation Summarization & Persistance

Generate a structured summary of the entire interaction, ready for logging.

#### 4° LangSmith Integration

In [47]:
# Test Query

# Urgent-Negative: 
# "This is an emergency! My order #TEC-2024-001 never arrived and I need that laptop for work tomorrow!"

test_queries[1]

'Urgent-Negative: This is an emergency! My order #TEC-2024-001 never arrived and I need that laptop for work tomorrow!'